In [ ]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [ ]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
caffe_root = '/home/ubuntu/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
import os
os.chdir('/home/ubuntu/deep-visualization-toolbox/models/my-model')
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

import io
io.stdout = sys.stdout = sys.__stdout__

In [ ]:
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
### load the solver and create train and test nets
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
solver = caffe.SGDSolver('sigmoid_solver.prototxt')
pretrained_model = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
solver.net.copy_from(pretrained_model)

In [ ]:
def set_ratio(ratio):
    solver.net.params['scale1_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale1_2'][0].data[...] = ratio * 4
    solver.net.params['scale2_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale2_2'][0].data[...] = ratio * 4
    solver.net.params['scale3_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale3_2'][0].data[...] = ratio * 4
    solver.net.params['scale4_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale4_2'][0].data[...] = ratio * 4
    solver.net.params['scale5_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale5_2'][0].data[...] = ratio * 4
    solver.net.params['scale6_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale6_2'][0].data[...] = ratio * 4
    solver.net.params['scale7_1'][0].data[...] = 1.0 / ratio
    solver.net.params['scale7_2'][0].data[...] = ratio * 4
set_ratio(800)

In [ ]:
print(solver.test_nets[0].params['scale1_1'][0].data[...])
print(solver.net.params['scale5_1'][0].data)
print(solver.net.params['bias5'][0].data)

In [ ]:
solver.net.forward()  # train net

In [ ]:
blobs = solver.net.blobs
print(blobs['conv1'].data[0, 0, 0, 0])
print(blobs['scale1_2'].data[0, 0, 0, 0])
print(blobs['conv2'].data[0, 0, 0, 0])
print(blobs['scale2_2'].data[0, 0, 0, 0])
print(blobs['conv3'].data[0, 0, 0, 0])
print(blobs['scale3_2'].data[0, 0, 0, 0])
print(blobs['conv4'].data[0, 0, 0, 0])
print(blobs['scale4_2'].data[0, 0, 0, 0])
print(blobs['conv5'].data[0, 0, 0, 0])
print(blobs['scale5_2'].data[0, 0, 0, 0])
print(blobs['fc6'].data[0, 0])
print(blobs['scale6_2'].data[0, 0])
print(blobs['fc7'].data[0, 0])
print(blobs['scale7_2'].data[0, 0])

In [ ]:
solver.test_nets[0].forward()  # test net (there can be more than one)

In [ ]:
transformer = caffe.io.Transformer({'data': solver.net.blobs['data'].data.shape})
data_mean = np.load('ilsvrc12/ilsvrc_2012_mean.npy').mean(1).mean(1)
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', data_mean - 8) # Subtract a bit more to avoid overflow，　only for visualizatio
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

In [ ]:
lookup_file = 'ilsvrc12/synset_words.txt'
lines = open(lookup_file).readlines()
names = [line.split()[1] for line in lines]

In [ ]:
for index in range(0, 10):
    print(names[int(solver.net.blobs['label'].data[index])])
    plt.imshow(np.clip(transformer.deprocess('data', solver.net.blobs['data'].data[index]), 0, 254))
    plt.show()

In [ ]:
solver.step(1)

In [ ]:
solver.net.blobs

In [ ]:
import time
niter = 20000
test_interval = 100
# losses will also be stored in the log
train_loss = np.zeros(niter)
test_acc = np.zeros(int(np.ceil(niter / test_interval)))
output = np.zeros((niter, 8, 10))

begin_time = time.time()
# the main solver loop
for it in range(niter):
    step_time = time.time()
    solver.step(1)  # SGD by Caffe
    elapsed_time = time.time() - step_time
    
    # store the train loss
    train_loss[it] = solver.net.blobs['loss'].data
    print("Iter " + str(it) + " -- Train loss: " + str(train_loss[it]) + 
          " -- Time used: " + str(elapsed_time) + "s -- Total time: " + str(time.time() - begin_time) + "s")

    # run a full test every so often
    # (Caffe can also do this for us and write to a log, but we show here
    #  how to do it directly in Python, where more complicated things are easier.)
    if it % test_interval == 0:
        print 'Iteration', it, 'testing...'
        correct = 0
        for test_it in range(100):
            solver.test_nets[0].forward()
            correct += sum(solver.test_nets[0].blobs['fc8'].data.argmax(1)
                           == solver.test_nets[0].blobs['label'].data)
        test_acc[it // test_interval] = correct / 1e4
        print("Test accuracy: " + str(correct / 1e4))

In [ ]:
solver.test_nets[0].blobs['data'].data.shape[0]